In [86]:
import os, sys, warnings
import concurrent.futures
from openpyxl import Workbook
from multiprocessing import Pool
from pathlib import Path
from IPython.display import Image, display
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2
#sys.path.insert(0, os.path.abspath(os.path.join(Path.home(), "rpa_libs")))

from selenium.webdriver.common.by import By 
from selenium.webdriver.common.keys import Keys

# Nossas libs
#import app
#from plugins.selenium.selenium import Selenium

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
# Exemplo de uso do arquivo de configuração
#app.redefine_config('caminho do seu arquivo .yaml')
#teste
import pandas as pd

In [59]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

In [66]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install().replace("THIRD_PARTY_NOTICES.", "")))

In [67]:
# Def the website i want to extract the table for
url = 'https://www.coingecko.com/pt'
driver.get(url)

In [100]:
# Picking table from Cripto Coins site and creating a variable for the rows in this table
table = driver.find_element(By.XPATH, '/html/body/div[2]/main/div/div[5]/table')
rows = table.find_elements(By.XPATH, './/tbody//tr')

In [101]:
# Creating the function to iterate in rows and cells of the table and appending to the table_data
def process_cells(cells):
    cell_data = []
    for cell in cells:
        img_tag = cell.find_elements(By.TAG_NAME, 'img')
        data_sort = cell.get_attribute('data-sort')
        if img_tag:
            # Extract the image URL (src attribute)
            img_src = img_tag[0].get_attribute('src')
            cell_data.append(img_src)
        if '$' in cell.text:
            cell_splited = cell.text.split(" ")
            for part_splitted in cell_splited:
                cell_data.append(part_splitted)
        elif data_sort:
            cell_data.append(data_sort)
        else:
            cell_data.append(cell.text)
    #table_data.append(cell_data)
    return cell_data

In [70]:
#with concurrent.futures.ThreadPoolExecutor() as executor:
#    results = list(executor.map(process_rows, rows))

#with Pool() as pool:
#    result = pool.map(process_rows, rows)

In [102]:
def process_rows(rows):
    table_data = []
    for row in rows[:10]:
    #for row in rows[:10]:
        cells = row.find_elements(By.XPATH, './/td')
        cells_data = process_cells(cells)
        table_data.append(cells_data)
    return table_data

In [103]:
table_data = process_rows(rows)

In [104]:
list(table_data)

[['',
  '1',
  'https://assets.coingecko.com/coins/images/1/standard/bitcoin.png?1696501400',
  'Bitcoin\nBTC',
  'Comprar',
  'US$',
  '63.358,23',
  '-0.34527838653457726',
  '0.04476426823681985',
  '6.572242965875268',
  '-0.2812308477441432',
  'US$',
  '28.605.925.824',
  'US$',
  '1.253.639.173.509',
  '1333610766004.817',
  '',
  'https://www.coingecko.com/coins/1/sparkline.svg',
  ''],
 ['',
  '2',
  'https://assets.coingecko.com/coins/images/279/standard/ethereum.png?1696501628',
  'Ethereum\nETH',
  'Comprar',
  'US$',
  '2.591,34',
  '-0.1753941029799257',
  '-1.2772161676601361',
  '13.108753634851029',
  '-4.578581093268481',
  'US$',
  '14.118.293.089',
  'US$',
  '312.857.111.944',
  '313186945426.0355',
  '',
  'https://www.coingecko.com/coins/279/sparkline.svg',
  ''],
 ['',
  '3',
  'https://assets.coingecko.com/coins/images/325/standard/Tether.png?1696501661',
  'Tether\nUSDT',
  'Comprar',
  'US$',
  '1,00',
  '-0.16257063201587',
  '-0.12629355661091152',
  '-0.14

In [143]:
headers = [header.text for header in table.find_elements(By.XPATH, './/thead//th')]
headers = [item for item in headers if item]
headers[0] = "IMG Coin"
headers.insert(0, "ID")
headers.insert(3, "$")
headers.insert(8, "30 d")
headers.insert(9, "$")
headers.insert(11, "$")
headers.insert(13, "FDV")
#table_data = [[cell for cell in row if cell.strip()] for row in table_data]

In [144]:
list(headers)

['ID',
 'IMG Coin',
 'Moeda',
 '$',
 'Preço',
 '1 h',
 '24 h',
 '7 d',
 '30 d',
 '$',
 'Volume em 24 h',
 '$',
 'Capitalização de mercado',
 'FDV',
 'Últimos 7 dias']

In [96]:
#list(table_data)

In [145]:
df = pd.DataFrame(table_data)
df_treated_with_link = df.loc[:, (df != '').any(axis=0)] #test drop_na
df_treated_with_link = df_treated_with_link.drop(df.columns[4], axis=1)


In [146]:
df_treated_with_link

,1,2,3,5,6,7,8,9,10,11,12,13,14,15,17
0,1,https://assets.coingecko.com/coins/images/1/st...,Bitcoin\nBTC,US$,"63.358,23",-0.34527838653457726,0.04476426823681985,6.572242965875268,-0.2812308477441432,US$,28.605.925.824,US$,1.253.639.173.509,1333610766004.817,https://www.coingecko.com/coins/1/sparkline.svg
1,2,https://assets.coingecko.com/coins/images/279/...,Ethereum\nETH,US$,"2.591,34",-0.1753941029799257,-1.2772161676601361,13.108753634851029,-4.578581093268481,US$,14.118.293.089,US$,312.857.111.944,313186945426.0355,https://www.coingecko.com/coins/279/sparkline.svg
2,3,https://assets.coingecko.com/coins/images/325/...,Tether\nUSDT,US$,"1,00",-0.16257063201587,-0.12629355661091152,-0.14456805539855858,-0.027952110980379342,US$,45.055.024.764,US$,119.124.974.151,119250563128.84433,https://www.coingecko.com/coins/325/sparkline.svg
3,4,https://assets.coingecko.com/coins/images/825/...,BNB\nBNB,US$,"590,51",-0.2775349361349781,-2.52103271606562,10.057746223511202,5.6910515878873245,US$,877.955.378,US$,86.532.056.517,86623284008.05759,https://www.coingecko.com/coins/825/sparkline.svg
4,5,https://assets.coingecko.com/coins/images/4128...,Solana\nSOL,US$,"150,50",0.23406210679963174,2.710081247141466,18.537797819805554,-4.2307282698188065,US$,3.401.606.505,US$,70.869.108.525,88576950732.02887,https://www.coingecko.com/coins/4128/sparkline...
5,6,https://assets.coingecko.com/coins/images/6319...,USDC\nUSDC,US$,"1,00",-0.1564808408078716,-0.17989331558015076,-0.16036717719360236,-0.12296175075617893,US$,6.943.096.671,US$,36.035.463.081,36077237887.332535,https://www.coingecko.com/coins/6319/sparkline...
6,7,https://assets.coingecko.com/coins/images/44/s...,XRP\nXRP,US$,"0,5838",-0.14956628552058343,-0.13596829705216695,3.260931371402314,-1.1285601830073713,US$,1.210.945.674,US$,33.078.006.648,58664690741.41599,https://www.coingecko.com/coins/44/sparkline.svg
7,8,https://assets.coingecko.com/coins/images/1344...,Lido Staked Ether\nSTETH,US$,"2.590,02",-0.15886399227116355,-1.2962529450928841,12.938684831876134,-4.481836395093942,US$,31.912.060,US$,25.433.474.759,25454527476.118233,https://www.coingecko.com/coins/13442/sparklin...
8,9,https://assets.coingecko.com/coins/images/5/st...,Dogecoin\nDOGE,US$,"0,1094",0.4518026937250138,1.5336048493310959,8.971413348668095,2.3010726725019746,US$,784.202.137,US$,16.039.414.158,16056616236.222075,https://www.coingecko.com/coins/5/sparkline.svg
9,10,https://assets.coingecko.com/coins/images/1798...,Toncoin\nTON,US$,"5,64",0.31254281268838907,2.781329233373763,4.5097674387629905,1.0911142661786788,US$,419.847.577,US$,14.420.029.001,29101561203.57846,https://www.coingecko.com/coins/17980/sparklin...


In [138]:
#df.style
#df.style.highlight_max()
#df.style.highlight_null(null_color="green")
#df.style.set_na_rep("OutofScope").highlight_null(null_color="orange")
#df.style.background_gradient()
#df.style.set_properties(**{'border': '1.3px solid black',
 #                         'color': 'white'})
#df.style.hide_index()

In [147]:
df_treated_with_link.style\
    .highlight_max(subset=[5], color='green')\
    .background_gradient(cmap='viridis', subset=[4])\
    .set_properties(**{'border': '1.3px solid black', 'color': 'white'})
df_treated_with_link = df_treated_with_link.reset_index(drop=True)

In [148]:
df_treated_with_link.columns = [headers]


In [149]:
df_treated.to_excel("file_test_df_cripto_coin.xlsx", index=False)

In [ ]:
for row in rows:
    cells = row.find_elements(By.XPATH, './/td')
    table_data.append([cell.text for cell in cells])

In [ ]:
for row in rows:
    cells = row.find_elements(By.XPATH, './/td')
    table_data.append([cell.text for cell in cells])